In [ ]:
from finrl.finrl_meta.env_cryptocurrency_trading.env_multiple_crypto import CryptoEnv
from finrl.agents.elegantrl.models import DRLAgent as DRLAgent_erl
from finrl.finrl_meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import datetime
from data_processor import config

import matplotlib



In [ ]:
import ccxt
import pandas as pd
import numpy as np


## Read Doneloaded Data

In [ ]:
tic_list = [
    "BNBUSDT",
    "BTCUSDT",
    "ETHUSDT",
    "XRPUSDT",
    "SOLUSDT",
    "LUNAUSDT",
    "ADAUSDT",
    "AVAXUSDT",
    "DOGEUSDT",
    "DOTUSDT",
]

interval = "1h"


In [ ]:
df_list = []
for tic in tic_list:
    data_path = f"./data/{tic}-{interval}.csv"
    data = pd.read_csv(data_path).values
    df = pd.DataFrame(
        data,
        columns=[
            "date",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_asset_volume",
            "number_of_trades",
            "taker_buy_base_asset_volume",
            "taker_buy_quote_asset_volume",
            "ignore",
        ],
    )
    df["date"] = pd.to_datetime(df["date"].add(1800000), unit="ms")
    df.drop(
        columns=[
            "close_time",
            "quote_asset_volume",
            "number_of_trades",
            "taker_buy_base_asset_volume",
            "taker_buy_quote_asset_volume",
            "ignore",
        ],
        inplace=True,
    )
    df["tic"] = tic
    df_list.append(df)

price_df = pd.concat(df_list)
price_df = price_df.sort_values(by=["date", "tic"], ignore_index=True)


In [ ]:
from data_processor.indicator import add_tech_indicator, add_fundamental_data

data_df = add_tech_indicator(price_df).dropna(how="any")
data_df.iloc[0]["date"]


In [ ]:
data_df = data_df.drop(columns=["open", "high", "low"])


In [ ]:
train = data_split(data_df, "2021-06-01", "2022-1-31")
trade = data_split(data_df, "2022-02-01", "2022-3-31")
print(len(train), len(trade))


In [ ]:
from my_trading_env.crypto_env import CryptoEnv
from my_trading_env.crypto_erl_agent import CryptoDRLAgent


In [ ]:
env = CryptoEnv(data_df=train, tech_indicators_list=config.TECH_INDICATORS, lookback=5)


In [ ]:
agent = CryptoDRLAgent(env)


In [ ]:
model_kwargs = {
    "learning_rate": 5e-6,
    "batch_size": 2**10,
    "gamma": 0.99,
    "seed": 312,
    "net_dimension": 2**12,
    "target_step": env.max_step * 2,
    "eval_gap": 50,
    "eval_times": 4,
}

model = agent.get_model("sac", model_kwargs)


In [ ]:
trained_model = agent.train_model(model=model, cwd='./test', total_timesteps=4e5)

## Backtest

In [ ]:
test_env = CryptoEnv(trade, config.TECH_INDICATORS)
test_result, actions = agent.DRL_prediction("sac", "./test", 2**12, test_env)


In [ ]:
actions[0:2]

In [ ]:
baseline_df = pd.DataFrame()
baseline_df['account_value'] = trade[trade['tic'] == "BTCUSDT"]['close'].to_list()[1:]
baseline_df['date'] = test_result['date']

In [ ]:
trade_ret = get_daily_return(test_result)
base_ret = get_daily_return(baseline_df)

In [ ]:
import pyfolio
%matplotlib inline

pyfolio.create_full_tear_sheet(trade_ret, benchmark_rets=base_ret)